In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#解压 补全缺失文件
import zipfile
import os
from tqdm import tqdm


# Define zip_path and extract_path here or ensure they are defined before this cell

#zip_path = '/content/drive/MyDrive/Colab Notebooks/dl/lecture12-2/words_captcha.zip'
#extract_path = '/content/drive/MyDrive/Colab Notebooks/dl/lecture12-2/'


os.makedirs(extract_path, exist_ok=True)

# Get the list of missing files from the previous cell's output
# Assuming 'missing_files' variable is available from the previous cell execution
if 'missing_files' not in locals() or not missing_files:
    print("No missing files found or missing_files list is not available. Skipping extraction.")
else:
    print(f"Attempting to extract {len(missing_files)} missing files.")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Create a set of missing files for efficient lookup
            missing_files_set = set(missing_files)

            # Get the list of files in the zip archive
            zip_file_list = zip_ref.namelist()

            # Filter the zip file list to include only missing files
            files_to_extract = [file for file in zip_file_list if os.path.basename(file) in missing_files_set]

            for file in tqdm(files_to_extract, desc="Extracting missing files"):
                # Construct the full path where the file should be extracted
                # Ensure the path within the zip file is handled correctly relative to extract_path
                # If the zip contains files directly at the root, just join extract_path and the filename
                # If the zip contains a folder structure, adjust accordingly
                file_dest_path = os.path.join(extract_path, os.path.basename(file))

                # Check again just in case to avoid redundant extraction if a file appeared between checks
                if not os.path.exists(file_dest_path):
                    # Extract the specific file by its name within the zip
                    # zip_ref.extract expects the path within the zip and an optional destination dir
                    # To extract a specific file to a specific location, we need to handle the path.
                    # A simpler approach for flat structures is to extract all and overwrite/skip existing,
                    # but the user asked to only extract missing.
                    # For this structure (a*.png directly in zip), we can extract to extract_path
                    zip_ref.extract(file, extract_path)


        print(f"\nExtraction complete. Missing files extracted to: {extract_path}")

    except FileNotFoundError:
        print(f"\nError: Zip file not found at {zip_path}")
    except KeyboardInterrupt:
        print("\nExtraction interrupted by user. Some files may not have been extracted.")
    except Exception as e:
        print(f"\nAn error occurred during extraction: {e}")

In [ ]:
#检查缺失文件
import os

extract_path = '/content/drive/MyDrive/Colab Notebooks/dl/lecture12-2/words_captcha'
expected_files = [f'a{i}.png' for i in range(0, 140000)]

existing_files = set(os.listdir(extract_path))

missing_files = [f for f in expected_files if f not in existing_files]

if missing_files:
    print(f"Found {len(missing_files)} missing files:")
    # Print only the first 100 missing files if there are many
    for i, missing_file in enumerate(missing_files):
        if i < 100:
            print(missing_file)
        else:
            print("...")
            break
else:
    print("No missing files found. All expected files are present.")

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

下载和准备数据集

In [ ]:
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [ ]:
import glob

DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/dl/lecture12-2/words_captcha/'  # 定义路径变量，便于维护

img_name_list = []
cap_list = []

with open(f'/content/drive/MyDrive/Colab Notebooks/dl/lecture12-2/spec_train_val.txt') as fin:  # 修改 txt 文件路径
    for line in fin:
        image_name, caption = line.strip().split()
        img_name_list.append(f'{DATA_PATH}{image_name}.png')  # 修改图像路径
        cap_list.append('<start> ' + ' '.join(caption) + ' <end>')

test_img_name = set(glob.glob(f'{DATA_PATH}*.png')) - set(img_name_list)  # 修改 glob 路径
img_name_list += sorted(test_img_name)

print(img_name_list[0])
print(cap_list[0])

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',
                                                  oov_token='')
tokenizer.fit_on_texts(cap_list)
cap_seqs = tokenizer.texts_to_sequences(cap_list)

cap_seqs = tf.keras.preprocessing.sequence.pad_sequences(cap_seqs, padding='post')
max_length = len(cap_seqs[0])

print(cap_list[0])
print(cap_seqs[0])


In [ ]:
img_name_train, img_name_valid = img_name_list[:100000], img_name_list[100000:120000]
cap_seqs_train, cap_seqs_valid = cap_seqs[:100000], cap_seqs[100000:]

img_name_test = img_name_list[120000:]

In [ ]:
IMAGE_SIZE = (160, 300)
BATCH_SIZE = 50
BUFFER_SIZE = 5000

EMBEDDING_DIM = 256
UNITS = 512
VOCAB_SIZE = len(tokenizer.word_index) + 1
EPOCHS = 10
STEPS = len(img_name_train) // BATCH_SIZE
LEARNING_RATE = 1e-4

building data

In [ ]:
def map_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255 * 2 - 1
    return img, img_path

def map_train(img_path, caption):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255 * 2 - 1 # Normalize to [-1, 1]
    return img, caption

dataset_train = tf.data.Dataset.from_tensor_slices((img_name_train, cap_seqs_train))\
                               .map(map_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                               .shuffle(BUFFER_SIZE)\
                               .batch(BATCH_SIZE, drop_remainder=True)\
                               .prefetch(tf.data.experimental.AUTOTUNE)

dataset_valid = tf.data.Dataset.from_tensor_slices((img_name_valid, cap_seqs_valid))\
                               .map(map_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                               .batch(BATCH_SIZE)\
                               .prefetch(tf.data.experimental.AUTOTUNE)

Build Model

In [ ]:
from tensorflow.keras import layers


class conv_leaky_relu(layers.Layer):
    def __init__(self, filters, size, stride):
        super(conv_leaky_relu, self).__init__()
        self.conv_2d = layers.Conv2D(filters, size, stride, padding='same')
        self.batch_norm = layers.BatchNormalization()
        self.leakey_relu = layers.LeakyReLU(0.1)

    def call(self, inputs):
        x = self.conv_2d(inputs)
        x = self.batch_norm(x)
        x = self.leakey_relu(x)
        return x


In [ ]:
from tensorflow.keras import Input, layers, Model

inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = conv_leaky_relu(64, 7, 2)(inputs)
x = layers.MaxPool2D()(x)
x = conv_leaky_relu(192, 3, 1)(x)
x = layers.MaxPool2D()(x)
x = conv_leaky_relu(128, 1, 1)(x)
x = conv_leaky_relu(256, 3, 1)(x)
x = conv_leaky_relu(256, 1, 1)(x)
x = conv_leaky_relu(512, 3, 1)(x)
x = layers.MaxPool2D()(x)
x = conv_leaky_relu(256, 1, 1)(x)
x = conv_leaky_relu(512, 3, 1)(x)
x = conv_leaky_relu(256, 1, 1)(x)
x = conv_leaky_relu(512, 3, 1)(x)
x = conv_leaky_relu(256, 1, 1)(x)
x = conv_leaky_relu(512, 3, 1)(x)
x = conv_leaky_relu(256, 1, 1)(x)
x = conv_leaky_relu(512, 3, 1)(x)
x = conv_leaky_relu(512, 1, 1)(x)
x = conv_leaky_relu(1024, 3, 1)(x)
x = layers.MaxPool2D()(x)
x = conv_leaky_relu(512, 1, 1)(x)
x = conv_leaky_relu(1024, 3, 1)(x)
x = conv_leaky_relu(512, 1, 1)(x)
x = conv_leaky_relu(1024, 3, 1)(x)
x = conv_leaky_relu(1024, 3, 1)(x)
x = conv_leaky_relu(1024, 3, 2)(x)
x = conv_leaky_relu(1024, 3, 1)(x)
outputs = conv_leaky_relu(1024, 3, 1)(x)

feature_extractor = Model(inputs=inputs, outputs=outputs, name='YOLO')

feature_extractor.summary()



In [ ]:
class CNN_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 15, embedding_dim)
        # hidden shape == (batch_size, hidden_size)

        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, 15, 1)
        score = self.V(tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, 15, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector == (batch_size, embedding_dim)
        context_vector = tf.reduce_sum(attention_weights * features, axis=1)

        return context_vector, attention_weights

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # x shape == (batch_size, 1)
        # features shape == (batch_size, 15, embedding_dim)
        # hidden shape == (batch_size, hidden_size)

        # context_vector shape == (batch_size, embedding_dim)
        # attention_weights shape == (batch_size, 15, 1)
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + embedding_dim)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        # Try unpacking more values to see what GRU returns
        gru_outputs = self.gru(x)
        # Assuming it returns more than 2, let's try unpacking the first two
        output, state = gru_outputs[0], gru_outputs[1] # Access elements by index


        # x shape == (batch_size, 1, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
encoder = CNN_Encoder(EMBEDDING_DIM)
decoder = RNN_Decoder(EMBEDDING_DIM, UNITS, VOCAB_SIZE)

training model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_path = './checkpoints/YOLO/'
ckpt = tf.train.Checkpoint(feature_extractor=feature_extractor,
                           encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [ ]:
#@tf.function # Temporarily commented out for debugging
def train_step(img_tensor, target):
    batch_size = img_tensor.shape[0]
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * batch_size, 1)
    hidden = decoder.reset_state(batch_size=batch_size)

    loss = 0
    with tf.GradientTape() as tape:
        features = feature_extractor(img_tensor)
        # Explicitly calculate dimensions for reshape
        feature_shape = tf.shape(features)
        features = tf.reshape(features, (feature_shape[0], feature_shape[1] * feature_shape[2], feature_shape[3]))
        features = encoder(features)


        for i in range(1, target.shape[1]):
            predictions, hidden, attention_weights = decoder(dec_input, features, hidden) # Unpack all three returned values
            loss += loss_function(target[:, i], predictions)
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    mean_loss = (loss / tf.cast(target.shape[1], tf.float32)) # Cast to float for division

    trainable_variables = feature_extractor.trainable_variables + \
        encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return mean_loss

In [ ]:
def predict(img_tensor):
    batch_size = img_tensor.shape[0]
    dec_input = tf.expand_dims(
        [tokenizer.word_index['<start>']] * batch_size, 1)

    features = feature_extractor(img_tensor)
    features = tf.reshape(features, (features.shape[0], -1, features.shape[3]))
    features = encoder(features)

    hidden = decoder.reset_state(batch_size=batch_size)

    result = tf.expand_dims([tokenizer.word_index['<start>']] * batch_size, 1)
    for _ in range(max_length):
        predictions, hidden, _ = decoder(dec_input, features, hidden)
        predicted_id = tf.argmax(predictions, axis=1).numpy()
        dec_input = tf.expand_dims(predicted_id, 1)
        result = tf.concat([result, predicted_id.reshape((batch_size, 1))], axis=1)

    return result

In [ ]:
def postprocess(segs):
    result_list = []
    for seq in segs:
        result = ''
        for s in seq[1:]:
            if s == tokenizer.word_index['<end>']:
                break
            result += tokenizer.index_word[s]
        result_list.append(result)
    return result_list

In [ ]:
def evaluate(dataset_valid):
    sample_count = 0
    correct_count = 0
    for img_tensor, target in dataset_valid:
        pred_list = postprocess(predict(img_tensor).numpy())
        real_list = postprocess(target.numpy())

        for pred, real in zip(pred_list, real_list):
            sample_count += 1
            if pred == real:
                correct_count += 1

    return correct_count / sample_count

In [ ]:
import time
from tqdm import tqdm

loss_plot = []
start = time.time()
for epoch in range(start_epoch, EPOCHS):
    loss = 0
    pbar = tqdm(dataset_train, total=STEPS, desc=f'Epoch {epoch + 1:2d}')
    for (step, (img_tensor, target)) in enumerate(pbar):
        loss += train_step(img_tensor, target)
        pbar.set_postfix({'loss': loss.numpy() / (step + 1)})

    loss_plot.append(loss / STEPS)
    ckpt_manager.save()

    score = evaluate(dataset_valid)
    print(f'Validation accuracy: {score:.2f}')

print('Time taken for {} epoch {} sec\\n'.format(EPOCHS - start_epoch, time.time() - start))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

Predict Testing Data

In [ ]:
ckpt = tf.train.Checkpoint(feature_extractor=feature_extractor,
                           encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt.restore('./checkpoints/YOLO/ckpt-10')

In [ ]:
score = evaluate(dataset_valid)
print(f'Validation accuracy: {score:.2f}')

In [ ]:
def map_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255 * 2 - 1
    return img, img_path

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((img_name_test))\
                              .map(map_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                              .batch(100)\
                              .prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
import re

with open('./Lab12-2_110062802.txt', 'w') as fout:
    for step, (img_tensor, img_path) in enumerate(tqdm(dataset_test)):
        pred_list = postprocess(predict(img_tensor).numpy())
        for path, pred in zip(img_path, pred_list):
            path = path.numpy().decode('utf-8')
            name = re.search('(a[0-9]+)', path).group(1)
            fout.write(f'{name} {pred}\\n')